# Error mitigation

This notebook contains examples for using our Error Mitigation module with Qiskit.

## Theoretical background

In this section description of proposed method will be provided. There are several important things to note first. Our method is targeted at such error model wherein dominant noise is classical. In practice, alongiside classical noise, there also exists coherent noise. Information about both of these noises can be, in a way, extracted from QDT. The usage of QDT module has been extracted in previous section.

Let $M$ denote POVM describing noisy detector and $P$ denote ideal measurement. Then classical error can be mathematically expressed as a stochastic, invertable map $\Lambda$ in such way, that:

$$ M = \Lambda P.$$

If that's that case, then following correction procedure can be applied. Imagine, we've access to experimental statistics $p_{exp}$ and we want to correct them in such way, that would allow us to obtain ideal one $p_{ideal}$. Due to the assumed nature of the noise, we can write, that:

$$ p_{exp} = \Lambda p_{ideal}. $$

We've assumed, when describing the noise, that $\Lambda$ is invertible. If so, then by multiplying last equation by $\Lambda^{-1}$ from both sides, we obtain

$$ \Lambda^{-1} p_{exp} = p_{ideal}. $$

Effectively, by this kind of postprocessing, we obtain statistics close to what we would normally obtain on non-noisy devices.

## Error mitigator

